In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

api_key='fernando'
sparql = SPARQLWrapper("http://sparql.bioontology.org/sparql")
sparql.addCustomParameter("apikey", api_key)
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX snomed-term: <http://purl.bioontology.org/ontology/SNOMEDCT/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX food: <http://data.lirmm.fr/ontologies/food#>

    SELECT DISTINCT ?allergy_label ?allergy ?allergenic ?allergenic_label
    WHERE
    {   
        {
            ?allergy rdfs:subClassOf snomed-term:419199007 .
            ?allergy skos:prefLabel ?allergy_label .
            ?allergy <http://purl.bioontology.org/ontology/SNOMEDCT/has_causative_agent> ?allergenic .
            ?allergenic skos:prefLabel ?allergenic_label.
        }
        union
        {
            ?allergy rdfs:subClassOf ?a.
            ?a rdfs:subClassOf snomed-term:419199007 .
            ?allergy <http://purl.bioontology.org/ontology/SNOMEDCT/has_causative_agent> ?allergenic .
            ?allergy skos:prefLabel ?allergy_label .
            ?allergenic skos:prefLabel ?allergenic_label.
        }
        union
        {
            ?allergy rdfs:subClassOf ?b .
            ?b rdfs:subClassOf ?a .
            ?a rdfs:subClassOf snomed-term:419199007 .
            ?allergy <http://purl.bioontology.org/ontology/SNOMEDCT/has_causative_agent> ?allergenic .
            ?allergy skos:prefLabel ?allergy_label .
            ?allergenic skos:prefLabel ?allergenic_label.
        }
        union
        {
            ?allergy rdfs:subClassOf ?c .
            ?c rdfs:subClassOf ?b .
            ?b rdfs:subClassOf ?a .
            ?a rdfs:subClassOf snomed-term:419199007 .
            ?allergy <http://purl.bioontology.org/ontology/SNOMEDCT/has_causative_agent> ?allergenic .
            ?allergy skos:prefLabel ?allergy_label .
            ?allergenic skos:prefLabel ?allergenic_label.
        }
        union
        {
            ?allergy rdfs:subClassOf ?d .
            ?d rdfs:subClassOf ?c .
            ?c rdfs:subClassOf ?b .
            ?b rdfs:subClassOf ?a .
            ?a rdfs:subClassOf snomed-term:419199007 .
            ?allergy <http://purl.bioontology.org/ontology/SNOMEDCT/has_causative_agent> ?allergenic .
            ?allergy skos:prefLabel ?allergy_label .
            ?allergenic skos:prefLabel ?allergenic_label.
        }
        union
        {
            ?allergy rdfs:subClassOf ?e .
            ?e rdfs:subClassOf ?d .
            ?d rdfs:subClassOf ?c .
            ?c rdfs:subClassOf ?b .
            ?b rdfs:subClassOf ?a .
            ?a rdfs:subClassOf snomed-term:419199007 .
            ?allergy <http://purl.bioontology.org/ontology/SNOMEDCT/has_causative_agent> ?allergenic .
            ?allergy skos:prefLabel ?allergy_label .
            ?allergenic skos:prefLabel ?allergenic_label.
        }
        FILTER()
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
        print('{}\t{}'.format(result["allergy_label"]["value"],result["allergenic_label"]["value"]))


In [ ]:
results['results']['bindings'][0]['allergy_label'].keys()

In [ ]:
%endpoint http://localhost:8890/sparql
# %display raw
%format json
%qparam apikey f9367108-21b4-4554-aa43-dee8fb7ccaea                             

%show all
PREFIX food: <http://data.lirmm.fr/ontologies/food#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dbpedia: <http://dbpedia.org/ontology/> 

SELECT ?name ?ingredients
WHERE {
        ?product food:IngredientListAsText ?ingredients .
        ?product food:name ?name .
        minus {?product food:IngredientListAsText ""}
}

In [ ]:
def query_ingredient(ingredient):
    
    sparql = SPARQLWrapper("http://localhost:8890/sparql")
    sparql.setQuery("""
        PREFIX food: <http://data.lirmm.fr/ontologies/food#>
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX dbpedia: <http://dbpedia.org/ontology/> 

        SELECT ?name ?ingredients
        WHERE {
                ?product food:IngredientListAsText ?ingredients .
                ?product food:name ?name .
                FILTER regex(str(lcase(?ingredients)), '%s')
        }
    """ % ingredient)
    sparql.setReturnFormat(JSON)
    results_food = sparql.query().convert()
    return results_food


In [ ]:
def query_dish(dish_name):
    
    sparql = SPARQLWrapper("http://localhost:8890/sparql")
    sparql.setQuery("""
        PREFIX food: <http://data.lirmm.fr/ontologies/food#>
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX dbpedia: <http://dbpedia.org/ontology/> 

        SELECT ?name ?ingredients
        WHERE {
                ?product food:IngredientListAsText ?ingredients .
                ?product food:name ?name .
                FILTER regex(str(lcase(?name)), '%s')
        }
    """ % dish_name)
    sparql.setReturnFormat(JSON)
    results_food = sparql.query().convert()
    return results_food

In [ ]:
results_food = query_dish('ham sandwich')

In [ ]:
results_food = query_ingredient('chayote')

In [ ]:
len(results_food['results']['bindings'])

In [ ]:
ingredients = set()
for i in results_food['results']['bindings']:
    print(i['ingredients']['value'])
    for j in i['ingredients']['value'].replace('(',',').replace(')',',').split(','):
        ingredients.add(j.lower().lstrip().rstrip())
try:
    ingredients.remove('.')
    ingredients.remove('')
except:
    pass

In [ ]:
ingredients